In [12]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 50)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from google.cloud import bigquery
from google.oauth2 import service_account
from pathlib import Path
from sklearn import set_config; set_config(display='diagram')
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer



In [2]:
def get_data_from_bq(
    gcp_project: str,
    query: str):
    client = bigquery.Client()
    query_job = client.query(query)
    result = query_job.result()
    df = result.to_dataframe()
    return df

GCP_PROJECT = "train-delays-406412"
GCP_REGION = "europe-west1"
BQ_REGION = "EU"
BQ_DATASET = "train_delays"

In [14]:
df = get_data_from_bq(gcp_project=GCP_PROJECT, query='SELECT * FROM `train-delays-406412.train_delays.train_delays_cleaned`')
df

,TRUST_TRAIN_ID_AFFECTED,PLANNED_ORIG_WTT_DATETIME_AFF,PLANNED_DEST_WTT_DATETIME_AFF,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,TRACTION_TYPE_AFFECTED,UNIT_CLASS_AFFECTED,RESPONSIBLE_MANAGER,INCIDENT_REASON,REACTIONARY_REASON_CODE,PERFORMANCE_EVENT_CODE,PFPI_MINUTES,Lat_OR,Lon_OR,STATION_OR,Lat_DES,Lon_DES,STATION_DES
0,522N48MJ08,2018-12-08 11:45:00,2018-12-08 12:47:00,22214000,EK01,SA,Y,EMU,375.0,MEKJ,MD,YI,M,9.0,51.54343,-0.02447,HACKNEY WICK,51.54343,-0.02447,HACKNEY WICK
1,522Y72MF19,2022-12-19 09:36:00,2022-12-19 10:39:00,22214000,EK01,WD,Y,EMU,378.0,MEKJ,MD,YI,M,12.0,51.54343,-0.02447,HACKNEY WICK,51.54343,-0.02447,HACKNEY WICK
2,522N20MZ30,2023-01-30 20:17:00,2023-01-30 21:19:00,22214000,EK01,WD,Y,EMU,378.0,XQHM,X8,YI,M,8.0,51.54343,-0.02447,HACKNEY WICK,51.54343,-0.02447,HACKNEY WICK
3,522N02MX24,2018-11-24 18:33:00,2018-11-24 19:32:00,22214000,EK01,SA,Y,EMU,375.0,TEKA,TG,YI,M,10.0,51.54343,-0.02447,HACKNEY WICK,51.54343,-0.02447,HACKNEY WICK
4,522Y08MX24,2018-11-24 18:53:00,2018-11-24 19:55:00,22214000,EK01,SA,Y,EMU,375.0,TEKA,TG,YI,M,11.0,51.54343,-0.02447,HACKNEY WICK,51.54343,-0.02447,HACKNEY WICK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498747,725J992823,2023-01-23 22:21:00,2023-01-23 22:55:00,21921000,EK01,WD,N,None,NaN,XQHM,XL,YD,M,12.0,51.54014,0.07965,BARKING NO7 PLATFORM,51.51911,0.11477,None
498748,515J04M513,2023-03-13 05:40:00,2023-03-13 05:49:00,21921000,EK01,WD,Y,EMU,710.0,TEKD,T3,YO,M,12.0,51.54014,0.07965,BARKING NO7 PLATFORM,51.51911,0.11477,None
498749,722J75MQ09,2023-05-09 15:24:00,2023-05-09 16:02:00,21921000,EK01,WD,Y,EMU,710.0,MEKP,M1,None,M,18.0,51.54014,0.07965,BARKING NO7 PLATFORM,51.51911,0.11477,None
498750,722J57MM30,2023-01-30 13:09:00,2023-01-30 13:47:00,21921000,EK01,WD,Y,EMU,710.0,DPEC,FX,YD,M,1.5,51.54014,0.07965,BARKING NO7 PLATFORM,51.51911,0.11477,None


In [15]:
df.loc[:, 'TRAIN_SERVICE_CODE_AFFECTED'] = df['TRAIN_SERVICE_CODE_AFFECTED'].astype(str)
df.loc[:, 'UNIT_CLASS_AFFECTED'] = df['UNIT_CLASS_AFFECTED'].astype(str)
df = df[df['TRACTION_TYPE_AFFECTED'] == 'EMU']
# df = df.drop(columns='TRUST_TRAIN_ID_AFFECTED')

In [16]:
df.loc[:, 'PLANNED_ORIG_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_ORIG_WTT_DATETIME_AFF'])
df.loc[:, 'ORI_MONTH'] = df['PLANNED_ORIG_WTT_DATETIME_AFF'].dt.month
df.loc[:, 'ORI_DAY'] = df['PLANNED_ORIG_WTT_DATETIME_AFF'].dt.day
df.loc[:, 'ORI_HOUR'] = df['PLANNED_ORIG_WTT_DATETIME_AFF'].dt.hour
df.loc[:, 'ORI_MINUTE'] = df['PLANNED_ORIG_WTT_DATETIME_AFF'].dt.minute
# df.drop(columns='PLANNED_ORIG_WTT_DATETIME_AFF', inplace = True)

/tmp/ipykernel_13116/3607659028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'PLANNED_ORIG_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_ORIG_WTT_DATETIME_AFF'])
/tmp/ipykernel_13116/3607659028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'ORI_MONTH'] = df['PLANNED_ORIG_WTT_DATETIME_AFF'].dt.month
/tmp/ipykernel_13116/3607659028.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [17]:
df.loc[:, 'PLANNED_DEST_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_DEST_WTT_DATETIME_AFF'])
df.loc[:, 'DES_MONTH'] = df['PLANNED_DEST_WTT_DATETIME_AFF'].dt.month
df.loc[:, 'DES_DAY'] = df['PLANNED_DEST_WTT_DATETIME_AFF'].dt.day
df.loc[:, 'DES_HOUR'] = df['PLANNED_DEST_WTT_DATETIME_AFF'].dt.hour
df.loc[:, 'DES_MINUTE'] = df['PLANNED_DEST_WTT_DATETIME_AFF'].dt.minute
# df.drop(columns='PLANNED_DEST_WTT_DATETIME_AFF', inplace = True)

/tmp/ipykernel_13116/807585888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'PLANNED_DEST_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_DEST_WTT_DATETIME_AFF'])
/tmp/ipykernel_13116/807585888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'DES_MONTH'] = df['PLANNED_DEST_WTT_DATETIME_AFF'].dt.month
/tmp/ipykernel_13116/807585888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [21]:
df = df[~df['INCIDENT_REASON'].isnull()]
df.loc[:, 'INCIDENT_REASON'] = df['INCIDENT_REASON'].apply(lambda x: x[0])

/tmp/ipykernel_13116/3783457221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'INCIDENT_REASON'] = df['INCIDENT_REASON'].apply(lambda x: x[0])


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494279 entries, 0 to 498751
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   TRUST_TRAIN_ID_AFFECTED        494279 non-null  object        
 1   PLANNED_ORIG_WTT_DATETIME_AFF  494279 non-null  datetime64[ns]
 2   PLANNED_DEST_WTT_DATETIME_AFF  494279 non-null  datetime64[ns]
 3   TRAIN_SERVICE_CODE_AFFECTED    494279 non-null  object        
 4   SERVICE_GROUP_CODE_AFFECTED    494279 non-null  object        
 5   ENGLISH_DAY_TYPE               494279 non-null  object        
 6   APP_TIMETABLE_FLAG_AFF         494279 non-null  object        
 7   TRACTION_TYPE_AFFECTED         494279 non-null  object        
 8   UNIT_CLASS_AFFECTED            494279 non-null  object        
 9   RESPONSIBLE_MANAGER            494279 non-null  object        
 10  INCIDENT_REASON                494279 non-null  object        
 11  

In [28]:
df.UNIT_CLASS_AFFECTED

0         375.0
1         378.0
2         378.0
3         375.0
4         375.0
          ...  
498746    710.0
498748    710.0
498749    710.0
498750    710.0
498751    710.0
Name: UNIT_CLASS_AFFECTED, Length: 494279, dtype: object

In [33]:
num_transformer = MinMaxScaler()

cat_transformer = OneHotEncoder(handle_unknown='ignore', drop='if_binary')



def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['Lat_OR', 'Lon_OR', 'Lat_DES', 'Lon_DES']),
    ('cat_transformer', cat_transformer, ['TRAIN_SERVICE_CODE_AFFECTED', 'SERVICE_GROUP_CODE_AFFECTED', 'ENGLISH_DAY_TYPE', 'APP_TIMETABLE_FLAG_AFF', 'UNIT_CLASS_AFFECTED', 'INCIDENT_REASON' ]),
    ('month_sin', sin_transformer(12), ['ORI_MONTH', 'DES_MONTH']),
    ('month_cos', cos_transformer(12), ['ORI_MONTH', 'DES_MONTH']),
    ('day_sin', sin_transformer(31), ['ORI_DAY', 'DES_DAY']),
    ('day_cos', cos_transformer(31), ['ORI_DAY', 'DES_DAY']),
    ('hour_sin', sin_transformer(24), ['ORI_HOUR', 'DES_HOUR']),
    ('hour_cos', cos_transformer(24), ['ORI_HOUR', 'DES_HOUR']),
    ('minute_sin', sin_transformer(60), ['ORI_MINUTE', 'ORI_MINUTE']),
    ('minute_cos', cos_transformer(60), ['ORI_MINUTE', 'ORI_MINUTE'])
], sparse_threshold=0, remainder='drop')

In [ ]:
num_transformer = MinMaxScaler()

cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output = False)

transformer = make_column_transformer((num_transformer, ['Lat_OR','Lon_OR', 'Lat_DES','Lon_DES']),
                                  (cat_transformer, ['ENGLISH_DAY_TYPE', 'SERVICE_GROUP_CODE_AFFECTED', 'INCIDENT_REASON',
                                                        'UNIT_CLASS_AFFECTED', 'TRAIN_SERVICE_CODE_AFFECTED', 
                                                     'PERFORMANCE_EVENT_CODE', 
                                                     'APP_TIMETABLE_FLAG_AFF']),
                                remainder = 'passthrough')


pipe = Pipeline([('transformer', transformer)])

pipe

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='PFPI_MINUTES'), df['PFPI_MINUTES'], test_size=0.2)

In [42]:
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)


In [43]:
X_train_scaled[0]

array([ 7.01158464e-01,  3.61215265e-01,  7.01158464e-01,  3.61215265e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -2.44929360e-16, -

In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

model_lin = LinearRegression()

cross_val_score(model_lin, X_train_scaled, y_train, cv=10, scoring='neg_root_mean_squared_error').mean()

-5.015501210437767

In [47]:
model_lin.fit(X_train_scaled, y_train)
model_lin.score(X_test_scaled, y_test)

0.09252795816403747

In [50]:
model_xgb = XGBRegressor(max_depth=10, n_estimators=500, learning_rate=0.01, n_jobs = -1)
cross_val_score(model_xgb, X_train_scaled, y_train, cv=10, scoring='neg_root_mean_squared_error').mean()

-4.421001051850099

In [51]:
model_xgb.fit(X_train_scaled, y_train)
model_xgb.score(X_test_scaled, y_test)

0.3193204809009539